# Libraries

In [21]:
import pandas as pd
import requests
import time
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup as bs
from io import StringIO

# Scraping the top 100 songs

I gathered the Billboard data by scraping the Billboard Year-End Hot 100 singles from Wikipedia for the years 1946 to 2023. The URLs I used follow this format: https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_XXXX, where ‘XXXX’ is the year. A couple of things to keep in mind about this dataset: for 1946 to 1948, each position has two numbers labeled “No.” and “Rank.” For example, in 1947, it looks like this:

```
| No. (Rank) | Title                | Artist(s)     |
-----------------------------------------------------
| 1 (1)      | "Near You"           | Francis Craig |
...
| 11 (12)    | "Managua, Nicaragua" | Freddy Martin |
```

I’m not entirely sure what those two numbers mean, so to make things easier, I just pulled the number in parentheses to use as the position. Also, the chart had 30 entries until 1955, then 50 until 1958, and since 1959, it’s had 100 entries. Sometimes, you’ll find two songs in the same position, like “Don’t” and “I Beg of You” by Elvis Presley in 1958. This happened because Billboard used to list two-sided hits as a single entry but later decided to list them separately [[1]](https://www.reddit.com/r/billboard/comments/12ooa7k/what_does_it_mean_when_two_songs_share_a_slot_on/). I noticed this issue later while pulling Spotify data, so I went through and cleaned things up manually.

In [22]:
# generate years
# years = [y for y in range(1946,2024)]
years = [y for y in range(2020,2024)]

# initial dataframe
charts_df = pd.DataFrame(columns=['rank', 'title', 'artists','year'])

for yr in years:
    
    # accessing the webpage and extracting the table
    time.sleep(.1)
    url = 'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_'+str(yr)
    r = requests.get(url)
    df_list = pd.read_html(StringIO(r.text)) # this parses all the tables in webpages to a list
    
    # the billboard table is either the first or second table in the webpage
    try:
        df = df_list[0]
        df['Title'] = df['Title'].str.extract(r'\"(.*)\"')
    except:
        df = df_list[1]
        df['Title'] = df['Title'].str.extract(r'\"(.*)\"')

    # extracting the song ranks/positions
    try:
        df['No. (Rank)'] = df['No. (Rank)'].str.extract(r'\((.*)\)')
    except:
        pass

    df['year'] = yr
    
    # updating the initial dataframe
    df.columns = charts_df.columns
    charts_df = pd.concat([charts_df, df], ignore_index=True)
    
    # printing the year obtained, for debugging purposes
    print(yr,end=' ')

charts_df.head(10)

2020 2021 2022 2023 

,rank,title,artists,year
0,1,Blinding Lights,The Weeknd,2020
1,2,Circles,Post Malone,2020
2,3,The Box,Roddy Ricch,2020
3,4,Don't Start Now,Dua Lipa,2020
4,5,Rockstar,DaBaby featuring Roddy Ricch,2020
5,6,Adore You,Harry Styles,2020
6,7,Life Is Good,Future featuring Drake,2020
7,8,Memories,Maroon 5,2020
8,9,The Bones,Maren Morris,2020
9,10,Someone You Loved,Lewis Capaldi,2020


# obtaining the spotify data

Enter the Spotify developer's client ID and client secret, which you can find on their website [[2]]( https://developer.spotify.com/)

In [23]:
# Load credentials from the configuration file
with open('spot_cred.json') as config_file:
    config = json.load(config_file)
    cid = config['client_id']
    csecret = config['client_secret']

client_credentials_manager = SpotifyClientCredentials(
    client_id=cid,
    client_secret=csecret

)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Obtaining song URIs

Next up, I needed to grab Spotify data for the songs, starting with their Spotify URIs, which are unique IDs for each track. I used Spotify's API to search for each song and pulled out the URI. This meant entering the song title and artist name, and usually, the correct song is the first one that pops up among the top ten search results. But sometimes, that top result isn’t right—it can end up being an instrumental or karaoke version instead.

To fix this, I tweaked the script to choose the first result that doesn’t have 'karaoke' or 'instrumental' in the title. Even with that change, a few wrong results still slipped through. A quick look online showed that a lot of other developers run into the same problem when using Spotify's API. For example, this discussion [[3]](https://github.com/spotipy-dev/spotipy/issues/858) talks about the same issue. To deal with these mix-ups, I ended up doing some manual data cleaning.

In [24]:
# a list of title-artist pairs
songartist = [(str(r['title']),str(r['artists'])) for i,r in charts_df.iterrows()]

# a list to store the search results
songuris = []
for song in songartist:
    while True:
        try:
            # the song title
            track = song[0]
            
            # the artists. i realised that the word 'featuring' can mess up the search results, so i just dropped it
            artist = song[1].replace('featuring','')
            s = sp.search(track+" "+artist)
            
            # the search results
            sres = s['tracks']['items']   
            
            # some placeholders for the results 
            res = 'unknown'
            sptr = 'unknown'
            spar = 'unknown'
            
            for r in sres:
                # only take the first record that doesn't contain certain terms
                if ('instrumental' not in r['name'].lower() and 'karaoke' not in r['name'].lower() and 'sped up' not in r['name'].lower()):
                    res = r['uri']
                    sptr = r['name']
                    spar = [k['name'] for k in r['artists']]
                    break
                    
            # structure and store
            d = {
                'uri': res,
                'wk_title': track,
                'wk_artist': artist,
                'sp_title': sptr,
                'sp_artist': spar
            }
            songuris.append(d)
            
            # print the result. for debugging purposes
            print(d)
            break
        
        # if failed, wait 10s and try again
        except Exception as e:
            print(f"Error occurred: {e}. Retrying in 10 seconds...")
            time.sleep(10)

{'uri': 'spotify:track:0VjIjW4GlUZAMYd2vXMi3b', 'wk_title': 'Blinding Lights', 'wk_artist': 'The Weeknd', 'sp_title': 'Blinding Lights', 'sp_artist': ['The Weeknd']}
{'uri': 'spotify:track:21jGcNKet2qwijlDFuPiPb', 'wk_title': 'Circles', 'wk_artist': 'Post Malone', 'sp_title': 'Circles', 'sp_artist': ['Post Malone']}
{'uri': 'spotify:track:0nbXyq5TXYPCO7pr3N8S4I', 'wk_title': 'The Box', 'wk_artist': 'Roddy Ricch', 'sp_title': 'The Box', 'sp_artist': ['Roddy Ricch']}
{'uri': 'spotify:track:3PfIrDoz19wz7qK7tYeu62', 'wk_title': "Don't Start Now", 'wk_artist': 'Dua Lipa', 'sp_title': "Don't Start Now", 'sp_artist': ['Dua Lipa']}
{'uri': 'spotify:track:7ytR5pFWmSjzHJIeQkgog4', 'wk_title': 'Rockstar', 'wk_artist': 'DaBaby  Roddy Ricch', 'sp_title': 'ROCKSTAR (feat. Roddy Ricch)', 'sp_artist': ['DaBaby', 'Roddy Ricch']}
{'uri': 'spotify:track:3jjujdWJ72nww5eGnfs2E7', 'wk_title': 'Adore You', 'wk_artist': 'Harry Styles', 'sp_title': 'Adore You', 'sp_artist': ['Harry Styles']}
{'uri': 'spotify:t

{'uri': 'spotify:track:7ju97lgwC2rKQ6wwsf9no9', 'wk_title': 'Rain on Me', 'wk_artist': 'Lady Gaga and Ariana Grande', 'sp_title': 'Rain On Me (with Ariana Grande)', 'sp_artist': ['Lady Gaga', 'Ariana Grande']}
{'uri': 'spotify:track:0PvFJmanyNQMseIFrU708S', 'wk_title': 'For the Night', 'wk_artist': 'Pop Smoke  Lil Baby and DaBaby', 'sp_title': 'For The Night (feat. Lil Baby & DaBaby)', 'sp_artist': ['Pop Smoke', 'Lil Baby', 'DaBaby']}
{'uri': 'spotify:track:6cy3ki60hLwimwIje7tALf', 'wk_title': 'Ritmo (Bad Boys for Life)', 'wk_artist': 'Black Eyed Peas and J Balvin', 'sp_title': 'RITMO (Bad Boys For Life)', 'sp_artist': ['Black Eyed Peas', 'J Balvin']}
{'uri': 'spotify:track:591ks2A0o7ratTpiRdI7M6', 'wk_title': 'Heart on Ice', 'wk_artist': 'Rod Wave', 'sp_title': 'Heart On Ice', 'sp_artist': ['Rod Wave']}
{'uri': 'spotify:track:4hbs0Y8EqyDBoA0G5aM1Z0', 'wk_title': 'Nobody but You', 'wk_artist': 'Blake Shelton and Gwen Stefani', 'sp_title': 'Nobody but You', 'sp_artist': ['Blake Shelton'

{'uri': 'spotify:track:0nGXi46VcQQ56ZJR428MKS', 'wk_title': 'Does to Me', 'wk_artist': 'Luke Combs  Eric Church', 'sp_title': 'Does To Me (feat. Eric Church)', 'sp_artist': ['Luke Combs', 'Eric Church']}
{'uri': 'spotify:track:48KXAIruJ07kJVCWOGohMV', 'wk_title': 'Pussy Fairy (OTW)', 'wk_artist': 'Jhené Aiko', 'sp_title': 'P*$$Y Fairy (OTW)', 'sp_artist': ['Jhené Aiko']}
{'uri': 'spotify:track:62aP9fBQKYKxi7PDXwcUAS', 'wk_title': 'ILY (I Love You Baby)', 'wk_artist': 'Surf Mesa  Emilee', 'sp_title': 'ily (i love you baby) (feat. Emilee)', 'sp_artist': ['Surf Mesa', 'Emilee']}
{'uri': 'spotify:track:5OELUCYgOHKFAvCERnAvfS', 'wk_title': 'More Than My Hometown', 'wk_artist': 'Morgan Wallen', 'sp_title': 'More Than My Hometown', 'sp_artist': ['Morgan Wallen']}
{'uri': 'spotify:track:0nYvjcSlCgjcwogQAwIwNp', 'wk_title': "Lovin' on You", 'wk_artist': 'Luke Combs', 'sp_title': "Lovin' On You", 'sp_artist': ['Luke Combs']}
{'uri': 'spotify:track:3sKz6Sd72K0ofPWcJPPk6H', 'wk_title': 'Said Sum',

{'uri': 'spotify:track:5CZ40GBx1sQ9agT82CLQCT', 'wk_title': 'Traitor', 'wk_artist': 'Olivia Rodrigo', 'sp_title': 'traitor', 'sp_artist': ['Olivia Rodrigo']}
{'uri': 'spotify:track:7FGdjbZ32vMybPuP0IuE7d', 'wk_title': 'Back in Blood', 'wk_artist': 'Pooh Shiesty  Lil Durk', 'sp_title': 'Back in Blood (feat. Lil Durk)', 'sp_artist': ['Pooh Shiesty', 'Lil Durk']}
{'uri': 'spotify:track:5cYFwgA5dHoqR3xGnlr9ew', 'wk_title': 'I Hope', 'wk_artist': 'Gabby Barrett  Charlie Puth', 'sp_title': 'I Hope (feat. Charlie Puth)', 'sp_artist': ['Gabby Barrett', 'Charlie Puth']}
{'uri': 'spotify:track:5QDLhrAOJJdNAmCTJ8xMyW', 'wk_title': 'Dynamite', 'wk_artist': 'BTS', 'sp_title': 'Dynamite', 'sp_artist': ['BTS']}
{'uri': 'spotify:track:0nqhKXDjsyBMvbeWmgijD0', 'wk_title': 'Wockesha', 'wk_artist': 'Moneybagg Yo', 'sp_title': 'Wockesha', 'sp_artist': ['Moneybagg Yo']}
{'uri': 'spotify:track:00Blm7zeNqgYLPtW6zg8cj', 'wk_title': 'You Right', 'wk_artist': 'Doja Cat and the Weeknd', 'sp_title': 'One Right No

{'uri': 'spotify:track:2S8UMuLJ0z8agFgFGJ6Ztt', 'wk_title': 'Good Time', 'wk_artist': 'Niko Moon', 'sp_title': 'GOOD TIME', 'sp_artist': ['Niko Moon']}
{'uri': 'spotify:track:7IYoh1u7dFM1xIWSEYd5B8', 'wk_title': "If I Didn't Love You", 'wk_artist': 'Jason Aldean and Carrie Underwood', 'sp_title': 'If I Didn’t Love You', 'sp_artist': ['Jason Aldean', 'Carrie Underwood']}
{'uri': 'spotify:track:2BcMwX1MPV6ZHP4tUT9uq6', 'wk_title': 'Knife Talk', 'wk_artist': 'Drake  21 Savage and Project Pat', 'sp_title': 'Knife Talk (with 21 Savage ft. Project Pat)', 'sp_artist': ['Drake', '21 Savage', 'Project Pat']}
{'uri': 'spotify:track:3UoULw70kMsiVXxW0L3A33', 'wk_title': 'POV', 'wk_artist': 'Ariana Grande', 'sp_title': 'pov', 'sp_artist': ['Ariana Grande']}
{'uri': 'spotify:track:4h5OVQQbeqw3lNaFIbmlzG', 'wk_title': 'Just the Way', 'wk_artist': 'Parmalee and Blanco Brown', 'sp_title': 'Just the Way', 'sp_artist': ['Parmalee', 'Blanco Brown']}
{'uri': 'spotify:track:2vgUijXOTRMnWXDtvgMp2b', 'wk_titl

{'uri': 'spotify:track:1nltpRhEiXikwDlVn4UADk', 'wk_title': 'One Right Now', 'wk_artist': 'Post Malone and the Weeknd', 'sp_title': 'One Right Now (with The Weeknd)', 'sp_artist': ['Post Malone', 'The Weeknd']}
{'uri': 'spotify:track:4ZtFanR9U6ndgddUvNcjcG', 'wk_title': 'Good 4 U', 'wk_artist': 'Olivia Rodrigo', 'sp_title': 'good 4 u', 'sp_artist': ['Olivia Rodrigo']}
{'uri': 'spotify:track:3o9kpgkIcffx0iSwxhuNI2', 'wk_title': 'Numb Little Bug', 'wk_artist': 'Em Beihold', 'sp_title': 'Numb Little Bug', 'sp_artist': ['Em Beihold']}
{'uri': 'spotify:track:3F5CgOj3wFlRv51JsHbxhe', 'wk_title': 'Jimmy Cooks', 'wk_artist': 'Drake  21 Savage', 'sp_title': 'Jimmy Cooks (feat. 21 Savage)', 'sp_artist': ['Drake', '21 Savage']}
{'uri': 'spotify:track:4k3lPl8YTKuY8c1HelVnm3', 'wk_title': "'Til You Can't", 'wk_artist': 'Cody Johnson', 'sp_title': "'Til You Can't", 'sp_artist': ['Cody Johnson']}
{'uri': 'spotify:track:58UKC45GPNTflCN6nwCUeF', 'wk_title': 'Fancy Like', 'wk_artist': 'Walker Hayes', 's

{'uri': 'spotify:track:4tYFy8ALRjIZvnvSLw5lxN', 'wk_title': 'Party', 'wk_artist': 'Bad Bunny and Rauw Alejandro', 'sp_title': 'Party', 'sp_artist': ['Bad Bunny', 'Rauw Alejandro']}
{'uri': 'spotify:track:1dm6z1fWB0cErMszU25dy2', 'wk_title': 'Después de la Playa', 'wk_artist': 'Bad Bunny', 'sp_title': 'Después de la Playa', 'sp_artist': ['Bad Bunny']}
{'uri': 'spotify:track:2UikqkwBv7aIvlixeVXHWt', 'wk_title': 'You Should Probably Leave', 'wk_artist': 'Chris Stapleton', 'sp_title': 'You Should Probably Leave', 'sp_artist': ['Chris Stapleton']}
{'uri': 'spotify:track:2EjXfH91m7f8HiJN1yQg97', 'wk_title': "Rockin' Around the Christmas Tree", 'wk_artist': 'Brenda Lee', 'sp_title': "Rockin' Around The Christmas Tree", 'sp_artist': ['Brenda Lee']}
{'uri': 'spotify:track:7qTaDOcld0VmBWXnkbUj45', 'wk_title': 'Broadway Girls', 'wk_artist': 'Lil Durk  Morgan Wallen', 'sp_title': 'Broadway Girls (feat. Morgan Wallen)', 'sp_artist': ['Lil Durk', 'Morgan Wallen']}
{'uri': 'spotify:track:7g0Zfd5J6Hzx

{'uri': 'spotify:track:3Qu3Zh4WTKhP9GEXo0aDnb', 'wk_title': 'Thought You Should Know', 'wk_artist': 'Morgan Wallen', 'sp_title': 'Thought You Should Know', 'sp_artist': ['Morgan Wallen']}
{'uri': 'spotify:track:1brnLTvarI9D1hLP6z2Ar8', 'wk_title': 'Thank God', 'wk_artist': 'Kane Brown and Katelyn Brown', 'sp_title': 'Thank God', 'sp_artist': ['Kane Brown', 'Katelyn Brown']}
{'uri': 'spotify:track:0JXXNGljqupsJaZsgSbMZV', 'wk_title': 'Sure Thing', 'wk_artist': 'Miguel', 'sp_title': 'Sure Thing', 'sp_artist': ['Miguel']}
{'uri': 'spotify:track:6HgWWaMu31KdOpEG5l28BG', 'wk_title': 'All My Life', 'wk_artist': 'Lil Durk  J. Cole', 'sp_title': 'All My Life (feat. J. Cole)', 'sp_artist': ['Lil Durk', 'J. Cole']}
{'uri': 'spotify:track:3dnP0JxCgygwQH9Gm7q7nb', 'wk_title': 'Ella Baila Sola', 'wk_artist': 'Eslabon Armado and Peso Pluma', 'sp_title': 'Ella Baila Sola', 'sp_artist': ['Eslabon Armado', 'Peso Pluma']}
{'uri': 'spotify:track:4i6cwNY6oIUU2XZxPIw82Y', 'wk_title': 'Karma', 'wk_artist': 

{'uri': 'spotify:track:0ZxhtATQ0a6EvQNZ0z6NKV', 'wk_title': 'Princess Diana', 'wk_artist': 'Ice Spice and Nicki Minaj', 'sp_title': 'Princess Diana (with Nicki Minaj)', 'sp_artist': ['Ice Spice', 'Nicki Minaj']}
{'uri': 'spotify:track:0WNfQxDGaPTl0yogcMR5v1', 'wk_title': 'Tomorrow 2', 'wk_artist': 'GloRilla and Cardi B', 'sp_title': 'Tomorrow 2 (with Cardi B)', 'sp_artist': ['GloRilla', 'Cardi B']}
{'uri': 'spotify:track:77khP2fIVhSW23NwxrRluh', 'wk_title': 'A Holly Jolly Christmas', 'wk_artist': 'Burl Ives', 'sp_title': 'A Holly Jolly Christmas', 'sp_artist': ['Burl Ives']}
{'uri': 'spotify:track:7ro0hRteUMfnOioTFI5TG1', 'wk_title': 'Where She Goes', 'wk_artist': 'Bad Bunny', 'sp_title': 'WHERE SHE GOES', 'sp_artist': ['Bad Bunny']}
{'uri': 'spotify:track:0IKeDy5bT9G0bA7ZixRT4A', 'wk_title': 'Bebe Dame', 'wk_artist': 'Fuerza Regida and Grupo Frontera', 'sp_title': 'Bebe Dame', 'sp_artist': ['Fuerza Regida', 'Grupo Frontera']}
{'uri': 'spotify:track:4KULAymBBJcPRpk1yO4dOG', 'wk_title':

Since this process took the longest, I decided to export the results into a CSV file for future use.

In [25]:
chartsURI = pd.concat([charts_df, pd.DataFrame(songuris)], axis=1)
arranged_df = chartsURI[['year', 'rank', 'uri', 'wk_title', 'wk_artist', 'sp_title', 'sp_artist']]
arranged_df.to_csv('annualbillboard_raw.csv', index=False)

## Manual data cleaning

I performed the data cleaning process using Google Sheets. I verified whether the song titles and artists matched the entries obtained from the Spotify API. This was done using various Google Sheets formulas, and most of the entries matched correctly. However, there were some discrepancies. For example, the Wikipedia entry listed the song as "Eastside" by Benny Blanco, Halsey, and Khalid, but the Spotify data returned "Eastside (8-Bit Benny Blanco & Halsey and Khalid Emulation)" by 8-Bit Arcade. To resolve such cases, I manually searched for the correct song on Spotify and copied the appropriate URI.

I also addressed issues with entries containing two songs in a single position. For example, in 1958, “Hard Headed Woman” / “Don’t Ask Me Why” was listed in position 49, but the data retrieved only included “Hard Headed Woman.” I manually added the URI for “Don’t Ask Me Why” from Spotify to ensure completeness. 

In total, I manually searched and updated URIs for 155 records, which is a manageable number given that the entire dataset consists of over 7,000 rows. There were 8 songs for which Spotify URIs were unavailable, so I marked these as "unknown" and excluded them from further analysis.

The cleaned dataset has been saved as a CSV file titled `annualbillboard.csv`.

## Obtaining song features

The song features provided by Spotify include acousticness, danceability, duration_ms, and many others. You can find a full description of these features on the Spotify developer website [[4]](https://developer.spotify.com/documentation/web-api/reference/get-audio-features).

In [26]:
# load the dataset
bb_df = pd.read_csv('annualbillboard.csv')
uris = bb_df['uri']

# we are allowed to obtain the features of 100 tracks at the same times.
fts = []
for i in range(0,len(uris),100):
    print(i,'-',i+100, end="; ")
    f = sp.audio_features(uris[i:i+100])
    fts+=f

0 - 100; 100 - 200; 200 - 300; 300 - 400; 400 - 500; 500 - 600; 600 - 700; 700 - 800; 800 - 900; 900 - 1000; 1000 - 1100; 1100 - 1200; 1200 - 1300; 1300 - 1400; 1400 - 1500; 1500 - 1600; 1600 - 1700; 1700 - 1800; 1800 - 1900; 1900 - 2000; 2000 - 2100; 2100 - 2200; 2200 - 2300; 2300 - 2400; 2400 - 2500; 2500 - 2600; 2600 - 2700; 2700 - 2800; 2800 - 2900; 2900 - 3000; 3000 - 3100; 3100 - 3200; 3200 - 3300; 3300 - 3400; 3400 - 3500; 3500 - 3600; 3600 - 3700; 3700 - 3800; 3800 - 3900; 3900 - 4000; 4000 - 4100; 4100 - 4200; 4200 - 4300; 4300 - 4400; 4400 - 4500; 4500 - 4600; 4600 - 4700; 4700 - 4800; 4800 - 4900; 4900 - 5000; 5000 - 5100; 5100 - 5200; 5200 - 5300; 5300 - 5400; 5400 - 5500; 5500 - 5600; 5600 - 5700; 5700 - 5800; 5800 - 5900; 5900 - 6000; 6000 - 6100; 6100 - 6200; 6200 - 6300; 6300 - 6400; 6400 - 6500; 6500 - 6600; 6600 - 6700; 6700 - 6800; 6800 - 6900; 6900 - 7000; 7000 - 7100; 

In [28]:
# turn the results into a dataframe. fill the unknown records with `None` rows
mt = {k: None for k in fts[0].keys()}
features_df = pd.DataFrame(data=[d if d is not None else mt for d in fts])

# combine the titles and artists dataframe with the feature dataframe
full_df = pd.concat([bb_df, features_df], axis=1)

# rearrange the columns and export to csv
arranged_df = full_df.iloc[:,[0,1,19,20,21,22,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,23,24]]
arranged_df.to_csv('billboard_features.csv', index=False)
arranged_df.head()

,year,rank,id,uri,track_href,analysis_url,wk_title,wk_artist,sp_title,sp_artist,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature
0,1946,1,3SpqiwLh1vaR0QeEDDvUrC,spotify:track:3SpqiwLh1vaR0QeEDDvUrC,https://api.spotify.com/v1/tracks/3SpqiwLh1vaR...,https://api.spotify.com/v1/audio-analysis/3Spq...,Prisoner of Love,Perry Como,Prisoner of Love (with Russ Case & His Orchest...,"['Perry Como', 'Russ Case & His Orchestra & Ch...",...,1.0,0.0333,0.911,0.000359,0.1190,0.191,80.938,audio_features,207200.0,4.0
1,1946,2,4xRdYnoXKNDPPrEicGPGCe,spotify:track:4xRdYnoXKNDPPrEicGPGCe,https://api.spotify.com/v1/tracks/4xRdYnoXKNDP...,https://api.spotify.com/v1/audio-analysis/4xRd...,To Each His Own,Eddy Howard,To Each His Own,['Eddy Howard'],...,1.0,0.0394,0.947,0.000060,0.1810,0.150,81.037,audio_features,184293.0,4.0
2,1946,3,4kuoSXnEyTKWypYkvjjnwd,spotify:track:4kuoSXnEyTKWypYkvjjnwd,https://api.spotify.com/v1/tracks/4kuoSXnEyTKW...,https://api.spotify.com/v1/audio-analysis/4kuo...,The Gypsy,The Ink Spots,The Gypsy,['The Ink Spots'],...,1.0,0.0552,0.812,0.000000,0.1030,0.222,76.056,audio_features,164533.0,4.0
3,1946,4,08jcVB28seFgQ5WrQiLKZQ,spotify:track:08jcVB28seFgQ5WrQiLKZQ,https://api.spotify.com/v1/tracks/08jcVB28seFg...,https://api.spotify.com/v1/audio-analysis/08jc...,Five Minutes More,Frank Sinatra,Five Minutes More,['Frank Sinatra'],...,1.0,0.0611,0.794,0.000000,0.0762,0.569,142.894,audio_features,154773.0,4.0
4,1946,5,7AoGplARvZSMmwYrsWjSfy,spotify:track:7AoGplARvZSMmwYrsWjSfy,https://api.spotify.com/v1/tracks/7AoGplARvZSM...,https://api.spotify.com/v1/audio-analysis/7AoG...,Rumors Are Flying,Frankie Carle,Rumors Are Flying,['Frankie Carle'],...,1.0,0.0437,0.951,0.002520,0.6380,0.377,72.355,audio_features,183133.0,4.0
